In [25]:
from populate_database import *
documents = load_documents()

In [26]:
# we know the first PDF has 16 pages, and the second has 116
#  so we can check if the loader splits the document up by page 
len(documents)

132

In [28]:
# what does the metadata look like per page?
print(documents[0].metadata)
print(documents[16].metadata)

{'source': 'data/Alaskas-Changing-Environment_2019_WEB.pdf', 'page': 0}
{'source': 'data/ArcticReportCard_full_report2024.pdf', 'page': 0}


In [31]:
print(documents[1].page_content)

Longest growing season
(155 days) Bethel, 2018
Warmest month
Y akutat, July 2019
Five warmest winters
Utqiaġvik, 2014–2019
Flooding
Dalton Highway 
April–May 2015
Record heat
Northway 92ºF 
June 2019
Record heat 90ºF 
Anchorage, July 2019
"Blob" marine heatwave
Gulf of Alaska, 2014–2016
Drought
Southeast Alaska,  
2017–2019
Warmest year 
Kodiak, 2016
Coastal flooding
Little Diomede 
February 2018
Record low sea ice extent 
Bering Sea, spring 
2018–2019
Warmest year
St. Paul, 2016
Earliest break-up
Tanana River 2019
Wildfire
Four million acres burned in 
five weeks, June–July 2015
1
Alaska has recently experienced profound environmental change related to extreme 
weather events and deviations from the historical climate. Sustained warmth, sea ice 
loss, coastal flooding, river flooding, and major ecosystem changes have impacted the 
daily lives of Alaskans around the state. 
Temperatures have been consistently warmer than at any time in the past century. This 
warming varies greatly acr

In [35]:
print(documents[91].page_content)

NOAA Technical Report OAR ARC ; 24-09  Arctic Report Card 2024 
74 
The neighboring boreal forest biome (see Fig. 1a,b) occupies large swaths of northern Eurasia and North 
America, and has also emerged as a focal point of global environmental change. Patches of positive and 
negative greenness trends are widely interspersed, reflecting complex interactions among the biome’s 
active wildfire regime, climate change, permafrost thaw, extreme events, pathogens, and other factors 
(Kim et al. 2024). Browning has generally prevailed in the warmer southern boreal zone (Berner and 
Goetz 2022), but greening has been widespread along the forest-tundra ecotone in the north, often in 
association with tall shrub and tree expansion (Wong et al. 2024). 
At the time of writing, MaxNDVI data for 2024 were only available for MODIS. In 2024, the MODIS 
observed circumpolar average MaxNDVI value was slightly higher (0.9%) compared to 2023 and 
represents the second highest in the 25-year record for tha

In [37]:
# possible to have a unique ID for document objects, but not required... here we didn't assign any
documents[0].id

In [40]:
# split the documents into chunks and see how many there are
chunks = split_documents(documents)
print(len(chunks))

429


In [43]:
# check out a chunk and its metadata
print(chunks[10].metadata)
print(chunks[10].page_content)

{'source': 'data/Alaskas-Changing-Environment_2019_WEB.pdf', 'page': 2}
1952 1960 1970 1980 1990 2000 2010 2019
40
30
20
10
0
Days per summer when visibility 
is restricted by smoke
BIG FIRE SEASONS MORE FREQUENT
Warmer springs and earlier snow melt had lengthened 
the wildfire season to the point in 2006 when Alaska’s 
interagency fire management organization changed the 
“start date” for wildfire response from May 1 to April 1. While 
the year-to-year variability of acreage burned has changed 
little, the frequency of large wildfire seasons has increased 
dramatically. Wildfire seasons with more than one million acres 
(red bars in graph) burned have increased by 50% since 1990, 
compared to the 1950–1989 period. 
Total acres burned each year
2
3
5
4
1
0.5
Average
-0.5
-1
1980 1985 1990 1995 2000 2005 2010 2015 2018


In [45]:
# just like the document objects, the chunks don't have unique IDs unless you assign them
chunks[10].id

In [48]:
# the functions in this codebase DO assign unique IDs to chunks based on document name, page number, and chunk number on that page
# this is done in order to make the database update-friendly
# during a database update, the unique IDs are used to ignore chunks that already exist
# but in this case, we add a unique ID to the chunk metadata dictionary
chunks_with_ids = calculate_chunk_ids(chunks)
print(chunks_with_ids[10].metadata["id"])

data/Alaskas-Changing-Environment_2019_WEB.pdf:2:3


In [ ]:
# So chunk #10 this is from the AK Changing Environment document, page 2, chunk 3